In [1]:
import pandas as pd
import numpy as np
import datetime as dt
import warnings as wn
import sklearn.preprocessing as skpre
import category_encoders as ce

wn.simplefilter( "ignore" )

In [2]:
loc_ftr = r"D:\TP2_ML\FeaturesLocal\FeaturesST"

In [3]:
cant_auc_trn = pd.read_csv(loc_ftr + "\\cant_auc_trn.csv")
cant_auc_tst = pd.read_csv(loc_ftr + "\\cant_auc_tst.csv")

cant_clk_trn = pd.read_csv(loc_ftr + "\\cant_clk_trn.csv")
cant_clk_tst = pd.read_csv(loc_ftr + "\\cant_clk_tst.csv")

cant_ins_trn = pd.read_csv(loc_ftr + "\\cant_ins_trn.csv")
cant_ins_tst = pd.read_csv(loc_ftr + "\\cant_ins_tst.csv")

cant_evt_trn = pd.read_csv(loc_ftr + "\\cant_evt_trn.csv")
cant_evt_tst = pd.read_csv(loc_ftr + "\\cant_evt_tst.csv")

cant_auc_trn.head(2)

,ref_hash,cant_auc
0,40621409780134,1
1,168103949904656,1


## <span style="color:yellow"> **=======================================================================================================** </span> 

## <span style="color:green"> **Cantidad de Clicks dividido la cantidad de subastas** </span>

In [23]:
#Sumo 1 a la cantidad de subastas para evitar dividir por 0
clk_auc_trn = cant_clk_trn.copy()
clk_auc_tst = cant_clk_tst.copy()

clk_auc_trn.columns = ['ref_hash','clk/auc']
clk_auc_tst.columns = ['ref_hash','clk/auc']

clk_auc_trn['clk/auc'] = clk_auc_trn['clk/auc'] / (cant_auc_trn['cant_auc'] + 1)
clk_auc_tst['clk/auc'] = clk_auc_tst['clk/auc'] / (cant_auc_tst['cant_auc'] + 1)

clk_auc_trn.to_csv( loc_ftr + "\\dv_cl_au_trn.csv", index = False )
clk_auc_tst.to_csv( loc_ftr + "\\dv_cl_au_tst.csv", index = False )

## <span style="color:green"> **Cantidad de instalaciones dividido la cantidad de subastas** </span>

In [24]:
#Sumo 1 a la cantidad de subastas para evitar dividir por 0
ins_auc_trn = cant_ins_trn.copy()
ins_auc_tst = cant_ins_tst.copy()

ins_auc_trn.columns = ['ref_hash','ins/auc']
ins_auc_tst.columns = ['ref_hash','ins/auc']

ins_auc_trn['ins/auc'] = ins_auc_trn['ins/auc'] / (cant_auc_trn['cant_auc'] + 1)
ins_auc_tst['ins/auc'] = ins_auc_tst['ins/auc'] / (cant_auc_tst['cant_auc'] + 1)

ins_auc_trn.to_csv( loc_ftr + "\\dv_in_au_trn.csv", index = False )
ins_auc_tst.to_csv( loc_ftr + "\\dv_in_au_tst.csv", index = False )

## <span style="color:green"> **Cantidad de eventos dividido la cantidad de subastas** </span>

In [25]:
#Sumo 1 a la cantidad de subastas para evitar dividir por 0
evt_auc_trn = cant_evt_trn.copy()
evt_auc_tst = cant_evt_tst.copy()

evt_auc_trn.columns = ['ref_hash','evt/auc']
evt_auc_tst.columns = ['ref_hash','evt/auc']

evt_auc_trn['evt/auc'] = evt_auc_trn['evt/auc'] / (cant_auc_trn['cant_auc'] + 1)
evt_auc_tst['evt/auc'] = evt_auc_tst['evt/auc'] / (cant_auc_tst['cant_auc'] + 1)

evt_auc_trn.to_csv( loc_ftr + "\\dv_ev_au_trn.csv", index = False )
evt_auc_tst.to_csv( loc_ftr + "\\dv_ev_au_tst.csv", index = False )

## <span style="color:green"> **Cantidad de sucesos del dispositivo** </span>

In [26]:
#Da una idea de la actividad total del dispositivo
sum_trn = cant_clk_trn.copy()
sum_tst = cant_clk_tst.copy()

sum_trn.columns = ['ref_hash','sum_cl_ev_in_au']
sum_tst.columns = ['ref_hash','sum_cl_ev_in_au']

sum_trn['clk/auc'] = sum_trn['sum_cl_ev_in_au'] + cant_evt_trn['cant_evt'] + cant_ins_trn['cant_ins'] + cant_auc_trn['cant_auc']
sum_tst['clk/auc'] = sum_tst['sum_cl_ev_in_au'] + cant_evt_tst['cant_evt'] + cant_ins_tst['cant_ins'] + cant_auc_tst['cant_auc']

sum_trn.to_csv( loc_ftr + "\\sum_ceia_trn.csv", index = False )
sum_tst.to_csv( loc_ftr + "\\sum_ceia_tst.csv", index = False )

# <span style="color:black"> **Importo eventos para proximos features** </span>

## <span style="color:yellow"> **Preparamos los Sets de Training** </span> 

In [4]:
loc_ts = r"D:\TP2_ML\Training Sets"

In [5]:
evt_trn = pd.read_csv( loc_ts + "\\evt_18_20.csv" ); 
evt_tst = pd.read_csv( loc_ts + "\\evt_21_23.csv" );  

## <span style="color:yellow"> **Preparamos el ref_hash de cada ventana** </span> 

In [6]:
loc_lb = r"D:\TP2_ML\Labels"

In [7]:
lb_auc_trn = pd.read_csv( loc_lb + "\\label_auc_21_23.csv" );
lb_auc_tst = pd.read_csv( loc_lb + "\\target.csv" ); 

In [8]:
rh_trn = lb_auc_trn[ ["ref_hash"] ]
rh_tst = lb_auc_tst[ ["ref_hash"] ]

tg_trn = lb_auc_trn[ ["21_23_st"] ]

In [9]:
evt_trn['date'] = pd.to_datetime(evt_trn['date'])
evt_tst['date'] = pd.to_datetime(evt_tst['date'])


## <span style="color:black"> **Mas features usando datos de eventos** </span>

## <span style="color:green"> **Cantidad de eventos antes de la primer subasta en la ventana** </span>

In [36]:
frst_auc_trn = pd.read_csv( loc_ftr + "\\frst_auc_trn.csv")
frst_auc_tst = pd.read_csv( loc_ftr + "\\frst_auc_tst.csv")

evt_loc_trn=evt_trn.loc[:,['ref_hash','date']]
evt_loc_tst=evt_trn.loc[:,['ref_hash','date']]

evt_loc_trn["time_to_frt_evt"] = ( pd.to_datetime( evt_loc_trn["date"] ) -  dt.datetime( year = 2019, month = 4, day = 18 ) ).dt.total_seconds()
evt_loc_tst["time_to_frt_evt"] = ( pd.to_datetime( evt_loc_tst["date"] ) -  dt.datetime( year = 2019, month = 4, day = 21 ) ).dt.total_seconds()

evt_loc_trn = evt_loc_trn.merge(frst_auc_trn, on = 'ref_hash', how = 'left')
evt_loc_tst = evt_loc_tst.merge(frst_auc_tst, on = 'ref_hash', how = 'left')

evt_loc_trn = evt_loc_trn.fillna(0)
evt_loc_tst = evt_loc_tst.fillna(0)

evt_loc_trn['cant_ev_before_auc'] = evt_loc_trn['time_to_frt_evt'] < evt_loc_trn['time_to_frt_auc']
evt_loc_tst['cant_ev_before_auc'] = evt_loc_tst['time_to_frt_evt'] < evt_loc_trn['time_to_frt_auc']

evt_loc_trn = evt_loc_trn.loc[evt_loc_trn['cant_ev_before_auc'] == True]
evt_loc_tst = evt_loc_tst.loc[evt_loc_tst['cant_ev_before_auc'] == True]

evt_loc_trn = evt_loc_trn.drop(['date','time_to_frt_evt','time_to_frt_auc'],axis=1)
evt_loc_tst = evt_loc_tst.drop(['date','time_to_frt_evt','time_to_frt_auc'],axis=1)

evt_loc_trn = evt_loc_trn.groupby('ref_hash').agg({'cant_ev_before_auc':'count'}).reset_index()
evt_loc_tst = evt_loc_tst.groupby('ref_hash').agg({'cant_ev_before_auc':'count'}).reset_index()

evt_bef_auc_trn = frst_auc_trn.merge(evt_loc_trn, on='ref_hash', how='left') 
evt_bef_auc_tst = frst_auc_tst.merge(evt_loc_tst, on='ref_hash', how='left') 

evt_bef_auc_trn = evt_bef_auc_trn.drop(['time_to_frt_auc'],axis=1)
evt_bef_auc_tst = evt_bef_auc_tst.drop(['time_to_frt_auc'],axis=1)

#Reemplazo los NANS con 0 (Son bastantes, no se si esta bien esto porque no es seguro que haya participado en al menos una subasta)
evt_bef_auc_trn = evt_bef_auc_trn.fillna(0)
evt_bef_auc_tst = evt_bef_auc_tst.fillna(0)

evt_bef_auc_trn.to_csv( loc_ftr + "\\ev_be_au_trn.csv", index = False )
evt_bef_auc_tst.to_csv( loc_ftr + "\\ev_be_au_tst.csv", index = False )

In [21]:
#len(frst_auc_trn.loc[frst_auc_trn['time_to_frt_auc'] < 259200])


326945

In [27]:
#frst_auc_trn.head()

,ref_hash,time_to_frt_auc
0,40621409780134,226857.222979
1,168103949904656,173044.526198
2,186034136943920,88.851558
3,295841792051458,432.827168
4,345999128501141,164703.549846


## <span style="color:green"> **Diferencia de tiempo entre primer subasta y proximo evento** </span>

In [13]:
frst_auc_trn = pd.read_csv( loc_ftr + "\\frst_auc_trn.csv")
frst_auc_tst = pd.read_csv( loc_ftr + "\\frst_auc_tst.csv")

evt_loc_trn=evt_trn.loc[:,['ref_hash','date']]
evt_loc_tst=evt_trn.loc[:,['ref_hash','date']]

evt_loc_trn["time_to_frt_evt"] = ( pd.to_datetime( evt_loc_trn["date"] ) -  dt.datetime( year = 2019, month = 4, day = 18 ) ).dt.total_seconds()
evt_loc_tst["time_to_frt_evt"] = ( pd.to_datetime( evt_loc_tst["date"] ) -  dt.datetime( year = 2019, month = 4, day = 21 ).dt.total_seconds()

evt_loc_trn = evt_loc_trn.merge(frst_auc_trn, on = 'ref_hash', how = 'left')
evt_loc_tst = evt_loc_tst.merge(frst_auc_tst, on = 'ref_hash', how = 'left')

evt_loc_trn = evt_loc_trn.fillna(259200.0)
evt_loc_tst = evt_loc_tst.fillna(259200.0)

evt_loc_trn['time_f_ev_aftr_auc'] = evt_loc_trn['time_to_frt_evt'] > evt_loc_trn['time_to_frt_auc']
evt_loc_tst['time_f_ev_aftr_auc'] = evt_loc_tst['time_to_frt_evt'] > evt_loc_trn['time_to_frt_auc']

evt_loc_trn = evt_loc_trn.loc[evt_loc_trn['time_f_ev_aftr_auc'] == True]
evt_loc_tst = evt_loc_tst.loc[evt_loc_tst['time_f_ev_aftr_auc'] == True]

evt_loc_trn['time_f_ev_aftr_auc'] = evt_loc_trn['time_to_frt_evt'] - evt_loc_trn['time_to_frt_auc']
evt_loc_tst['time_f_ev_aftr_auc'] = evt_loc_tst['time_to_frt_evt'] - evt_loc_trn['time_to_frt_auc']

evt_loc_trn = evt_loc_trn.groupby('ref_hash').agg({'time_f_ev_aftr_auc':'min'}).reset_index()
evt_loc_tst = evt_loc_tst.groupby('ref_hash').agg({'time_f_ev_aftr_auc':'min'}).reset_index()

time_f_ev_aftr_auc_trn = frst_auc_trn.merge(evt_loc_trn, on='ref_hash', how='left') 
time_f_ev_aftr_auc_tst = frst_auc_tst.merge(evt_loc_tst, on='ref_hash', how='left') 

time_f_ev_aftr_auc_trn = time_f_ev_aftr_auc_trn.drop(['time_to_frt_auc'],axis=1)
time_f_ev_aftr_auc_tst = time_f_ev_aftr_auc_tst.drop(['time_to_frt_auc'],axis=1)

#Reemplazo los NANS con el tiempo maximo 259200 (Son bastantes, no se si esta bien esto porque no es seguro que haya participado en al menos una subasta)
time_f_ev_aftr_auc_trn = time_f_ev_aftr_auc_trn.fillna(259200)
time_f_ev_aftr_auc_tst = time_f_ev_aftr_auc_tst.fillna(259200)

time_f_ev_aftr_auc_trn.to_csv( loc_ftr + "\\tevaf_au_trn.csv", index = False )
time_f_ev_aftr_auc_tst.to_csv( loc_ftr + "\\tevaf_au_tst.csv", index = False )

## <span style="color:green"> **Primer tipo de evento despues de primer subasta** </span>

#### <span style="color:orange"> **Mean Encoding** </span>

In [16]:
frst_auc_trn = pd.read_csv( loc_ftr + "\\frst_auc_trn.csv")
frst_auc_tst = pd.read_csv( loc_ftr + "\\frst_auc_tst.csv")

evt_loc_trn=evt_trn.loc[:,['ref_hash','date','kind']]
evt_loc_tst=evt_trn.loc[:,['ref_hash','date','kind']]

evt_loc_trn["time_to_frt_evt"] = ( pd.to_datetime( evt_loc_trn["date"] ) -  dt.datetime( year = 2019, month = 4, day = 18 ) ).dt.total_seconds()
evt_loc_tst["time_to_frt_evt"] = ( pd.to_datetime( evt_loc_tst["date"] ) -  dt.datetime( year = 2019, month = 4, day = 21 ) ).dt.total_seconds()

evt_loc_trn = evt_loc_trn.merge(frst_auc_trn, on = 'ref_hash', how = 'left')
evt_loc_tst = evt_loc_tst.merge(frst_auc_tst, on = 'ref_hash', how = 'left')

evt_loc_trn = evt_loc_trn.fillna(259200.0)
evt_loc_tst = evt_loc_tst.fillna(259200.0)

evt_loc_trn['time_f_ev_aftr_auc'] = evt_loc_trn['time_to_frt_evt'] > evt_loc_trn['time_to_frt_auc']
evt_loc_tst['time_f_ev_aftr_auc'] = evt_loc_tst['time_to_frt_evt'] > evt_loc_trn['time_to_frt_auc']

evt_loc_trn = evt_loc_trn.loc[evt_loc_trn['time_f_ev_aftr_auc'] == True]
evt_loc_tst = evt_loc_tst.loc[evt_loc_tst['time_f_ev_aftr_auc'] == True]

evt_loc_trn['time_f_ev_aftr_auc'] = evt_loc_trn.groupby('ref_hash')['time_to_frt_evt'].transform('min')
evt_loc_tst['time_f_ev_aftr_auc'] = evt_loc_tst.groupby('ref_hash')['time_to_frt_evt'].transform('min')

evt_loc_trn['kind'] = evt_loc_trn.groupby('kind')['ref_hash'].transform('count')
evt_loc_tst['kind'] = evt_loc_tst.groupby('kind')['ref_hash'].transform('count')

evt_loc_trn['time_f_ev_aftr_auc'] = (evt_loc_trn['time_f_ev_aftr_auc'] == evt_loc_trn['time_to_frt_evt'])
evt_loc_tst['time_f_ev_aftr_auc'] = (evt_loc_tst['time_f_ev_aftr_auc'] == evt_loc_tst['time_to_frt_evt'])

evt_loc_trn = evt_loc_trn.loc[evt_loc_trn['time_f_ev_aftr_auc'] == True]
evt_loc_tst = evt_loc_tst.loc[evt_loc_tst['time_f_ev_aftr_auc'] == True]

evt_loc_trn = evt_loc_trn.groupby('ref_hash').agg({'kind':'min'}).reset_index()
evt_loc_tst = evt_loc_tst.groupby('ref_hash').agg({'kind':'min'}).reset_index()

kind_f_ev_aftr_auc_trn = frst_auc_trn.merge(evt_loc_trn, on='ref_hash', how='left') 
kind_f_ev_aftr_auc_tst = frst_auc_tst.merge(evt_loc_tst, on='ref_hash', how='left') 

kind_f_ev_aftr_auc_trn = kind_f_ev_aftr_auc_trn.drop(['time_to_frt_auc'],axis=1)
kind_f_ev_aftr_auc_tst = kind_f_ev_aftr_auc_tst.drop(['time_to_frt_auc'],axis=1)

kind_f_ev_aftr_auc_trn['kind'] = ( kind_f_ev_aftr_auc_trn['kind'] / len(kind_f_ev_aftr_auc_trn) )
kind_f_ev_aftr_auc_tst['kind'] = ( kind_f_ev_aftr_auc_tst['kind'] / len(kind_f_ev_aftr_auc_tst) )

#Reemplazo los NANS
kind_f_ev_aftr_auc_trn = kind_f_ev_aftr_auc_trn.fillna( ( kind_f_ev_aftr_auc_trn['kind'].isnull().sum() / len(kind_f_ev_aftr_auc_trn) ) )
kind_f_ev_aftr_auc_tst = kind_f_ev_aftr_auc_tst.fillna( ( kind_f_ev_aftr_auc_tst['kind'].isnull().sum() / len(kind_f_ev_aftr_auc_tst) ) )

kind_f_ev_aftr_auc_trn.to_csv( loc_ftr + "\\kevaf_au_trn.csv", index = False )
kind_f_ev_aftr_auc_tst.to_csv( loc_ftr + "\\kevaf_au_tst.csv", index = False )

## <span style="color:green"> **Primer tipo de event id despues de primer subasta** </span>

#### <span style="color:orange"> **Mean Encoding** </span>

In [17]:
frst_auc_trn = pd.read_csv( loc_ftr + "\\frst_auc_trn.csv")
frst_auc_tst = pd.read_csv( loc_ftr + "\\frst_auc_tst.csv")

evt_loc_trn=evt_trn.loc[:,['ref_hash','date','event_id']]
evt_loc_tst=evt_trn.loc[:,['ref_hash','date','event_id']]

evt_loc_trn["time_to_frt_evt"] = ( pd.to_datetime( evt_loc_trn["date"] ) -  dt.datetime( year = 2019, month = 4, day = 18 ) ).dt.total_seconds()
evt_loc_tst["time_to_frt_evt"] = ( pd.to_datetime( evt_loc_tst["date"] ) -  dt.datetime( year = 2019, month = 4, day = 21 ) ).dt.total_seconds()

evt_loc_trn = evt_loc_trn.merge(frst_auc_trn, on = 'ref_hash', how = 'left')
evt_loc_tst = evt_loc_tst.merge(frst_auc_tst, on = 'ref_hash', how = 'left')

evt_loc_trn = evt_loc_trn.fillna(259200.0)
evt_loc_tst = evt_loc_tst.fillna(259200.0)

evt_loc_trn['time_f_ev_aftr_auc'] = evt_loc_trn['time_to_frt_evt'] > evt_loc_trn['time_to_frt_auc']
evt_loc_tst['time_f_ev_aftr_auc'] = evt_loc_tst['time_to_frt_evt'] > evt_loc_trn['time_to_frt_auc']

evt_loc_trn = evt_loc_trn.loc[evt_loc_trn['time_f_ev_aftr_auc'] == True]
evt_loc_tst = evt_loc_tst.loc[evt_loc_tst['time_f_ev_aftr_auc'] == True]

evt_loc_trn['time_f_ev_aftr_auc'] = evt_loc_trn.groupby('ref_hash')['time_to_frt_evt'].transform('min')
evt_loc_tst['time_f_ev_aftr_auc'] = evt_loc_tst.groupby('ref_hash')['time_to_frt_evt'].transform('min')

evt_loc_trn['event_id'] = evt_loc_trn.groupby('event_id')['ref_hash'].transform('count')
evt_loc_tst['event_id'] = evt_loc_tst.groupby('event_id')['ref_hash'].transform('count')

evt_loc_trn['time_f_ev_aftr_auc'] = (evt_loc_trn['time_f_ev_aftr_auc'] == evt_loc_trn['time_to_frt_evt'])
evt_loc_tst['time_f_ev_aftr_auc'] = (evt_loc_tst['time_f_ev_aftr_auc'] == evt_loc_tst['time_to_frt_evt'])

evt_loc_trn = evt_loc_trn.loc[evt_loc_trn['time_f_ev_aftr_auc'] == True]
evt_loc_tst = evt_loc_tst.loc[evt_loc_tst['time_f_ev_aftr_auc'] == True]

evt_loc_trn = evt_loc_trn.groupby('ref_hash').agg({'event_id':'min'}).reset_index()
evt_loc_tst = evt_loc_tst.groupby('ref_hash').agg({'event_id':'min'}).reset_index()

evid_f_ev_aftr_auc_trn = frst_auc_trn.merge(evt_loc_trn, on='ref_hash', how='left') 
evid_f_ev_aftr_auc_tst = frst_auc_tst.merge(evt_loc_tst, on='ref_hash', how='left') 

evid_f_ev_aftr_auc_trn = evid_f_ev_aftr_auc_trn.drop(['time_to_frt_auc'],axis=1)
evid_f_ev_aftr_auc_tst = evid_f_ev_aftr_auc_tst.drop(['time_to_frt_auc'],axis=1)

evid_f_ev_aftr_auc_trn['event_id'] = ( evid_f_ev_aftr_auc_trn['event_id'] / len(evid_f_ev_aftr_auc_trn) )
evid_f_ev_aftr_auc_tst['event_id'] = ( evid_f_ev_aftr_auc_tst['event_id'] / len(evid_f_ev_aftr_auc_tst) )

#Reemplazo los NANS
evid_f_ev_aftr_auc_trn = evid_f_ev_aftr_auc_trn.fillna( ( evid_f_ev_aftr_auc_trn['event_id'].isnull().sum() / len(evid_f_ev_aftr_auc_trn) ) )
evid_f_ev_aftr_auc_tst = evid_f_ev_aftr_auc_tst.fillna( ( evid_f_ev_aftr_auc_tst['event_id'].isnull().sum() / len(evid_f_ev_aftr_auc_tst) ) )

evid_f_ev_aftr_auc_trn.to_csv( loc_ftr + "\\eviaf_au_trn.csv", index = False )
evid_f_ev_aftr_auc_tst.to_csv( loc_ftr + "\\eviaf_au_tst.csv", index = False )